In [ ]:
from google.colab import files
import pandas as pd
import io

# Step 1: Upload the CSV file
uploaded = files.upload()

# Step 2: Print the keys to verify the uploaded file name
print(uploaded.keys())

# Step 3: Use the correct filename to read the CSV file
# Check if the expected file is in the uploaded dictionary
if 'train.csv' in uploaded:
    df1 = pd.read_csv(io.BytesIO(uploaded['train.csv']))
    print(df1.head())
else:
    print("File 'train.csv' not found. Please check the uploaded filenames.")


Saving train.csv to train.csv
dict_keys(['train.csv'])
  essay_id                                          full_text  score
0  000d118  Many people have car where they live. The thin...      3
1  000fe60  I am a scientist at NASA that is discussing th...      3
2  001ab80  People always wish they had the same technolog...      4
3  001bdc0  We all heard about Venus, the planet without a...      4
4  002ba53  Dear, State Senator\n\nThis is a letter to arg...      3


In [ ]:
import numpy as np
import pandas as pd
import io
import tensorflow as tf
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

In [ ]:
df1['processed_essay'] = df1['full_text'].apply(preprocess_text)
print(df1[['full_text', 'processed_essay']].head())

                                           full_text  \
0  Many people have car where they live. The thin...   
1  I am a scientist at NASA that is discussing th...   
2  People always wish they had the same technolog...   
3  We all heard about Venus, the planet without a...   
4  Dear, State Senator\n\nThis is a letter to arg...   

                                     processed_essay  
0  [many, people, car, live, thing, dont, know, u...  
1  [scientist, nasa, discussing, face, mar, expla...  
2  [people, always, wish, technology, seen, movie...  
3  [heard, venus, planet, without, almost, oxygen...  
4  [dear, state, senator, letter, argue, favor, k...  


In [ ]:
from gensim.models import Word2Vec
tokenized_texts = df1['processed_essay'].tolist()
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
def get_w2v_embeddings(text_tokens):
    embeddings = [w2v_model.wv[word] for word in text_tokens if word in w2v_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(w2v_model.vector_size)

In [ ]:
df1['w2v_embedding'] = df1['processed_essay'].apply(lambda x: get_w2v_embeddings(x))

In [ ]:
X = df1['processed_essay']
y = df1['score']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_pad.shape[1]))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Bidirectional(LSTM(units=64)))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/10
347/347 [==============================] - 1015s 3s/step - loss: 1.3785 - accuracy: 0.3985 - val_loss: 1.1330 - val_accuracy: 0.4511 - lr: 0.0010
Epoch 2/10
347/347 [==============================] - 939s 3s/step - loss: 0.9911 - accuracy: 0.5858 - val_loss: 1.0026 - val_accuracy: 0.5590 - lr: 0.0010
Epoch 3/10
347/347 [==============================] - 948s 3s/step - loss: 0.8071 - accuracy: 0.6799 - val_loss: 0.9805 - val_accuracy: 0.6009 - lr: 0.0010
Epoch 4/10
347/347 [==============================] - 941s 3s/step - loss: 0.6518 - accuracy: 0.7610 - val_loss: 1.0879 - val_accuracy: 0.5782 - lr: 0.0010
Epoch 5/10
347/347 [==============================] - 943s 3s/step - loss: 0.5274 - accuracy: 0.8172 - val_loss: 1.1377 - val_accuracy: 0.5789 - lr: 0.0010
Epoch 6/10
347/347 [==============================] - 926s 3s/step - loss: 0.3818 - accuracy: 0.8713 - val_loss: 1.3123 - val_accuracy: 0.5742 - lr: 0.0010
